# Group Project: 数据预加载&清洗

<hr>

In [1]:
import os
import re
import torch
import json
import numpy as np
import pandas as pd
import urllib.request
import tarfile
from pathlib import Path
from typing import List, Dict, Tuple
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import html

## DataDownloader - 数据下载与验证模块

**功能方法：**
- download_imdb() - 下载IMDb数据集压缩包并解压

- validate_data_directory() - 验证数据目录结构是否完整

- setup_data() - 主入口，确保数据可用（下载或使用现有数据）

<hr>

In [2]:
class DataDownloader:
    
    @staticmethod
    def download_imdb() -> Path:
        """下载IMDb数据集"""
        data_dir = Path('./aclImdb')
        
        if data_dir.exists():
            print("IMDb数据集已存在，跳过下载")
            return data_dir
        
        print("下载IMDb数据集...")
        url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
        tar_path = Path('./aclImdb_v1.tar.gz')
        
        try:
            # 下载文件
            urllib.request.urlretrieve(url, tar_path)
            print("下载完成，解压中...")
            
            # 解压文件
            with tarfile.open(tar_path, 'r:gz') as tar:
                tar.extractall(path='./')
            
            # 删除压缩包
            tar_path.unlink()
            print("IMDb数据集准备完成")
            
            return data_dir
        except Exception as e:
            raise Exception(f"数据集下载失败: {e}")
    
    @staticmethod
    def validate_data_directory(data_dir: Path) -> bool:
        """验证数据目录结构"""
        required_dirs = [
            data_dir / 'train' / 'pos',
            data_dir / 'train' / 'neg', 
            data_dir / 'test' / 'pos',
            data_dir / 'test' / 'neg'
        ]
        
        for dir_path in required_dirs:
            if not dir_path.exists():
                return False
            if len(list(dir_path.glob('*.txt'))) == 0:
                return False
                
        return True
    
    @staticmethod
    def setup_data() -> Path:
        """设置数据目录，确保数据可用"""
        data_dir = Path('./aclImdb')
        
        if DataDownloader.validate_data_directory(data_dir):
            print("✅ 数据目录验证通过")
            return data_dir
        
        print("数据目录不完整，重新下载...")
        data_dir = DataDownloader.download_imdb()
        
        if not DataDownloader.validate_data_directory(data_dir):
            raise Exception("数据目录验证失败，请手动检查数据集")
            
        return data_dir


## TextProcessor - 文本处理模块

**功能方法：**
- robust_text_cleaning(text) - 核心文本清洗，包含：

    1. HTML实体解码和标签移除
    
    2. URL处理
    
    3. 缩写词展开（如 "can't" → "can not"）
    
    4. 标点符号标准化
    
    5. 大小写统一

- process_in_batches() - 批量处理文本，避免内存溢出

<hr>

In [3]:
class TextProcessor:
    @staticmethod
    def robust_text_cleaning(text: str) -> str:
        """
        鲁棒的文本清洗管道 - 包含HTML处理
        """
        # 解码HTML实体
        text = html.unescape(text)
        
        # 移除HTML标签
        text = re.sub(r'<[^>]+>', ' ', text)
        
        # 处理URL
        text = re.sub(r'http\S+', ' <URL> ', text)
        
        # 处理常见的缩写和否定形式
        contractions = {
            r"won't": "will not", r"can't": "can not", r"n't": " not",
            r"'re": " are", r"'s": " is", r"'d": " would", 
            r"'ll": " will", r"'t": " not", r"'ve": " have",
            r"'m": " am"
        }
        
        for pattern, replacement in contractions.items():
            text = re.sub(pattern, replacement, text)
        
        # 保留基本的标点用于情感分析
        text = re.sub(r'[^a-zA-Z\s\.!?,;:\']', ' ', text)
        
        # 处理重复的标点
        text = re.sub(r'([!?.]){2,}', r'\1', text)
        text = re.sub(r'[!?.]+', r' \0 ', text)
        
        # 统一空格处理
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip().lower()
    
    @staticmethod
    def process_in_batches(texts: List[str], process_func, batch_size: int = 1000, process_name: str = "处理") -> List[str]:
        """批量处理文本数据"""
        processed = []
        total = len(texts)
        
        for i in range(0, total, batch_size):
            batch = texts[i:i + batch_size]
            processed_batch = [process_func(text) for text in batch]
            processed.extend(processed_batch)
            
            if i % 5000 == 0 and i > 0:
                print(f"  {process_name} {i}/{total} 个样本")
        
        print(f"✅ {process_name}完成")
        return processed

## VocabularyBuilder - 词汇表管理模块

**功能方法：**
- build_vocabulary(texts) - 基于训练文本构建词汇表：

    1. 统计词频
    
    2. 过滤低频词
    
    3. 添加特殊标记（PAD/UNK/BOS/EOS）
    
    4. 计算词汇表覆盖率

- text_to_sequence(text) - 将文本转换为数字序列：

    1. 添加开始/结束标记
    
    2. 处理未知词（OOV）
    
    3. 序列填充/截断
    
    4. 生成注意力掩码

<hr>

In [4]:
class VocabularyBuilder:
    
    def __init__(self, max_vocab_size: int = 30000, min_freq: int = 2):
        self.max_vocab_size = max_vocab_size
        self.min_freq = min_freq
        self.special_tokens = ['<PAD>', '<UNK>', '<BOS>', '<EOS>']
        self.token_to_id = {}
        self.id_to_token = {}
        self.vocab_size = 0
    
    def build_vocabulary(self, texts: List[str]) -> None:
        """构建词汇表"""
        print("构建词汇表...")
        
        # 统计词频
        word_freq = Counter()
        for text in texts:
            tokens = text.split()
            word_freq.update(tokens)
        
        # 过滤低频词
        filtered_words = [(word, freq) for word, freq in word_freq.items() 
                         if freq >= self.min_freq]
        
        # 按频率排序并选择前N个词
        sorted_words = sorted(filtered_words, key=lambda x: x[1], reverse=True)
        selected_words = [word for word, freq in sorted_words[:self.max_vocab_size - len(self.special_tokens)]]
        
        # 构建词汇表
        self.token_to_id = {}
        self.id_to_token = {}
        
        # 添加特殊标记
        for idx, token in enumerate(self.special_tokens):
            self.token_to_id[token] = idx
            self.id_to_token[idx] = token
        
        # 添加普通词汇
        for idx, word in enumerate(selected_words, start=len(self.special_tokens)):
            self.token_to_id[word] = idx
            self.id_to_token[idx] = word
        
        self.vocab_size = len(self.token_to_id)
        
        # OOV分析
        total_tokens = sum(word_freq.values())
        covered_tokens = sum(freq for word, freq in word_freq.items() 
                           if word in self.token_to_id)
        coverage = covered_tokens / total_tokens * 100
        
        print(f"词汇表构建完成: {self.vocab_size} 个token")
        print(f"词汇表覆盖率: {coverage:.2f}%")
    
    def text_to_sequence(self, text: str, max_length: int = 512, 
                        add_special_tokens: bool = True) -> Tuple[List[int], List[int], int]:
        """将文本转换为整数序列"""
        tokens = text.split()
        sequence = []
        
        # 添加开始标记
        if add_special_tokens:
            sequence.append(self.token_to_id['<BOS>'])
        
        # 转换tokens
        for token in tokens:
            sequence.append(self.token_to_id.get(token, self.token_to_id['<UNK>']))
        
        # 添加结束标记
        if add_special_tokens:
            sequence.append(self.token_to_id['<EOS>'])
        
        original_length = len(sequence)
        
        # 填充或截断
        if len(sequence) < max_length:
            sequence.extend([self.token_to_id['<PAD>']] * (max_length - len(sequence)))
            attention_mask = [1] * original_length + [0] * (max_length - original_length)
        else:
            sequence = sequence[:max_length]
            attention_mask = [1] * max_length
        
        return sequence, attention_mask, original_length

## IMDBDataPreprocessor - 数据预处理核心类

**功能方法：**

- load_raw_data() - 从文件系统加载原始IMDb数据

- split_data_with_seeds() - 数据划分为训练/验证/测试集（60%/20%/20%）

<hr>

In [5]:
import os
from pathlib import Path
from typing import List, Tuple
from collections import Counter

import numpy as np
import torch
from sklearn.model_selection import train_test_split

class IMDBDataPreprocessor:
    """
    IMDb数据预处理管道（星级标签版本 1-10）
    
    主要功能：
      - 加载IMDb原始数据，并将标签解析为1..10星级
      - 可复现的数据划分（训练/验证/测试），当某些类别样本过少时自动关闭分层抽样
      - 保持原有的VocabularyBuilder与TextProcessor初始化
    """

    def __init__(self, data_dir: str, max_vocab_size: int = 30000,
                 max_length: int = 512, min_freq: int = 2, seed: int = 42):
        self.data_dir = Path(data_dir)
        self.max_length = max_length
        self.seed = seed

        # 初始化组件
        self.vocab_builder = VocabularyBuilder(max_vocab_size, min_freq)
        self.text_processor = TextProcessor()

        # 设置随机种子
        torch.manual_seed(seed)
        np.random.seed(seed)

    def load_raw_data(self) -> Tuple[List[str], List[int], List[str], List[int]]:
        """
        加载原始IMDb数据，标签为1..10星级（注意：官方labeled split不包含5与6）
        返回：
            train_texts, train_labels, test_texts, test_labels
        """
        print("加载IMDb数据集...")

        def load_from_directory(directory: Path) -> Tuple[List[str], List[int]]:
            texts, labels = [], []
            for label_type in ['pos', 'neg']:
                dir_name = directory / label_type
                if not dir_name.exists():
                    print(f"Warning: 缺少目录: {dir_name}")
                    continue
                for file_path in dir_name.glob('*.txt'):
                    # 文件名格式: "<id>_<rating>.txt"，例如 "12345_7.txt"
                    stem = file_path.stem
                    parts = stem.split('_')
                    if len(parts) < 2:
                        print(f"跳过异常文件名: {file_path.name}")
                        continue
                    rating_str = parts[-1]
                    try:
                        rating = int(rating_str)  # 1..10
                    except ValueError as e:
                        print(f"跳过解析失败文件: {file_path.name} ({e})")
                        continue

                    # 读取文本
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            text = f.read().strip()
                    except Exception as e:
                        print(f"跳过无法读取文件: {file_path.name} ({e})")
                        continue

                    texts.append(text)
                    labels.append(rating)

            # 分布检查
            dist = Counter(labels)
            dist_sorted = dict(sorted(dist.items()))
            print(f"{directory.name} 星级分布: {dist_sorted}")
            return texts, labels

        train_dir = self.data_dir / 'train'
        test_dir = self.data_dir / 'test'

        train_texts, train_labels = load_from_directory(train_dir)
        test_texts, test_labels = load_from_directory(test_dir)

        print(f"加载了 {len(train_texts)} 训练样本, {len(test_texts)} 测试样本")
        return train_texts, train_labels, test_texts, test_labels

    def split_data_with_seeds(self, texts: List[str], labels: List[int],
                              test_size: float = 0.2, val_size: float = 0.2
                              ) -> Tuple[List[str], List[int], List[str], List[int], List[str], List[int]]:
        """
        使用随机种子进行可复现的数据划分（训练/验证/测试，60%/20%/20%）
        当任一类别样本数 < 2 时，自动关闭分层抽样以避免sklearn报错。
        """
        if len(texts) != len(labels):
            raise ValueError("texts 和 labels 长度不一致")

        def _print_dist(name: str, y: List[int]):
            c = Counter(y)
            print(f"{name} 类别分布: {dict(sorted(c.items()))}")

        print("执行可复现的数据划分（带类别检查）...")
        counts = Counter(labels)
        min_count = min(counts.values()) if counts else 0
        use_stratify = labels if min_count >= 2 else None
        if use_stratify is None:
            print(f"Warning: 最小类别样本数 {min_count} < 2，初次划分不使用分层抽样")

        # 先划分测试集
        train_texts, test_texts, train_labels, test_labels = train_test_split(
            texts, labels, test_size=test_size, random_state=self.seed,
            stratify=use_stratify
        )

        # 再从训练集划分验证集
        train_counts = Counter(train_labels)
        min_train_count = min(train_counts.values()) if train_counts else 0
        use_stratify_val = train_labels if min_train_count >= 2 else None
        if use_stratify_val is None:
            print(f"Warning: 训练集中最小类别样本数 {min_train_count} < 2，验证集划分不使用分层抽样")

        train_texts, val_texts, train_labels, val_labels = train_test_split(
            train_texts, train_labels, test_size=val_size, random_state=self.seed,
            stratify=use_stratify_val
        )

        print(f"数据划分: 训练集={len(train_texts)}, 验证集={len(val_texts)}, 测试集={len(test_texts)}")
        _print_dist("训练集", train_labels)
        _print_dist("验证集", val_labels)
        _print_dist("测试集", test_labels)

        return train_texts, train_labels, val_texts, val_labels, test_texts, test_labels          
    
### Updated for 1-10StarsVersion by Chen Yapeng end at this line

## IMDBDataset - 数据集类

**功能方法：**

- __init__() - 初始化数据集，验证数据一致性

- __len__() - 返回数据集大小

- __getitem__() - 获取单个样本，返回字典格式

<hr>

In [6]:
class IMDBDataset(Dataset):
    """IMDb数据集类"""
    
    def __init__(self, sequences: List[List[int]], attention_masks: List[List[int]], 
                 labels: List[int], lengths: List[int]):
        self.sequences = torch.tensor(sequences, dtype=torch.long)
        self.attention_masks = torch.tensor(attention_masks, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.lengths = torch.tensor(lengths, dtype=torch.long)
        
        # 验证数据形状
        assert len(self.sequences) == len(self.labels), "序列和标签数量不匹配"
        assert len(self.sequences) == len(self.attention_masks), "序列和注意力掩码数量不匹配"
        assert len(self.sequences) == len(self.lengths), "序列和长度数量不匹配"
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.sequences[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx],
            'lengths': self.lengths[idx]
        }

## DataSaver - 数据保存模块

**功能方法：**

- save_processed_data_multiclass() - 主保存函数

- _save_vocabulary() - 保存词汇表为JSON

- _save_config() - 保存预处理配置

- _save_tensor_data() - 保存Tensor数据为.pt文件

- _save_metadata() - 保存统计元数据

- _verify_files() - 验证生成的文件

<hr>

In [7]:
class DataSaver:

    @staticmethod
    def save_processed_data_multiclass(datasets: Dict, vocab, metadata: Dict, output_dir: str):
        """保存所有处理后的数据"""
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        print(f"开始保存数据到: {output_path}")
        
        # 1. 保存词汇表
        DataSaver._save_vocabulary(vocab, output_path)
        
        # 2. 保存配置
        DataSaver._save_config(output_path)
        
        # 3. 保存 tensor 数据
        DataSaver._save_tensor_data(datasets, output_path)
        
        # 4. 保存元数据
        DataSaver._save_metadata(metadata, output_path)
        
        # 5. 验证文件
        DataSaver._verify_files(output_path)
    
    @staticmethod
    def _save_vocabulary(vocab, output_path):
        """保存词汇表"""
        vocab_path = output_path / 'vocabulary.json'
        with open(vocab_path, 'w', encoding='utf-8') as f:
            json.dump(vocab, f, indent=2, ensure_ascii=False)
        print(f"✅ 词汇表保存到: {vocab_path}")
    
    @staticmethod
    def _save_config(output_path):
        """保存配置"""
        config_path = output_path / 'preprocessing_config.json'
        config = {
            'max_vocab_size': 30000,
            'max_length': 512,
            'min_freq': 2,
            'seed': 42,
            'saved_time': str(pd.Timestamp.now())
        }
        with open(config_path, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2)
        print(f"✅ 配置保存到: {config_path}")
    
    @staticmethod
    def _save_tensor_data(datasets, output_path):
        """保存 tensor 数据"""
        data_path = output_path / 'all_data.pt'
        
        save_data = {
            'train_sequences': datasets['train'].sequences,
            'train_masks': datasets['train'].attention_masks,
            'train_labels': datasets['train'].labels,
            'train_lengths': datasets['train'].lengths,
            
            'val_sequences': datasets['val'].sequences,
            'val_masks': datasets['val'].attention_masks,
            'val_labels': datasets['val'].labels,
            'val_lengths': datasets['val'].lengths,
            
            'test_sequences': datasets['test'].sequences,
            'test_masks': datasets['test'].attention_masks,
            'test_labels': datasets['test'].labels,
            'test_lengths': datasets['test'].lengths,
        }
        
        torch.save(save_data, data_path)
        file_size = data_path.stat().st_size / 1024 / 1024
        print(f"✅ Tensor数据保存到: {data_path} (大小: {file_size:.2f} MB)")
    
    @staticmethod
    def _save_metadata(metadata, output_path):
        """保存元数据"""
        metadata_path = output_path / 'preprocessing_metadata.json'
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, default=str, ensure_ascii=False)
        print(f"✅ 元数据保存到: {metadata_path}")
    
    @staticmethod
    def _verify_files(output_path):
        """验证文件是否创建成功"""
        print("\n📁 验证生成的文件:")
        for file in output_path.iterdir():
            size_kb = file.stat().st_size / 1024
            print(f"  {file.name} ({size_kb:.1f} KB)")

## DataProcessingPipeline - 主流水线

**功能方法：**

- run_pipeline() - 执行完整的数据处理流程（10个步骤）

- _vectorize_texts() - 批量向量化文本

- _create_datasets() - 创建训练/验证/测试数据集

<hr>

In [8]:
class DataProcessingPipeline:
    """完整的数据处理流水线"""
    
    def __init__(self, max_vocab_size: int = 30000, max_length: int = 512, 
                 min_freq: int = 2, seed: int = 42):
        self.max_vocab_size = max_vocab_size
        self.max_length = max_length
        self.min_freq = min_freq
        self.seed = seed
        
    def run_pipeline(self) -> bool:
        """运行完整的数据处理流水线"""
        try:
            # 1. 设置数据目录
            print("=== IMDb数据预处理开始 ===")
            data_dir = DataDownloader.setup_data()
            
            # 2. 初始化预处理器
            preprocessor = IMDBDataPreprocessor(
                data_dir=str(data_dir),
                max_vocab_size=self.max_vocab_size,
                max_length=self.max_length,
                min_freq=self.min_freq,
                seed=self.seed
            )
            
            # 3. 加载原始数据
            print("\n📥 加载完整原始数据...")
            train_texts, train_labels, test_texts, test_labels = preprocessor.load_raw_data()
            print(f"原始数据: {len(train_texts)} 训练样本, {len(test_texts)} 测试样本")
            
            # 4. 数据划分
            print("📊 完整数据划分...")
            all_texts = train_texts + test_texts
            all_labels = train_labels + test_labels
            
            train_texts, train_labels, val_texts, val_labels, test_texts, test_labels = \
                preprocessor.split_data_with_seeds(all_texts, all_labels)
            
            print(f"数据划分: 训练集={len(train_texts)}, 验证集={len(val_texts)}, 测试集={len(test_texts)}")
            
            # 5. 文本清洗
            print("🧹 清洗完整文本数据...")
            train_texts_clean = TextProcessor.process_in_batches(
                train_texts, TextProcessor.robust_text_cleaning, process_name="文本清洗"
            )
            val_texts_clean = TextProcessor.process_in_batches(
                val_texts, TextProcessor.robust_text_cleaning, process_name="文本清洗"
            )
            test_texts_clean = TextProcessor.process_in_batches(
                test_texts, TextProcessor.robust_text_cleaning, process_name="文本清洗"
            )
            
            # 6. 构建词汇表
            print("📚 构建完整词汇表...")
            preprocessor.vocab_builder.build_vocabulary(train_texts_clean)
            
            # 7. 文本向量化
            print("🔢 向量化完整数据...")
            train_sequences, train_masks, train_lengths = self._vectorize_texts(
                train_texts_clean, preprocessor.vocab_builder, preprocessor.max_length
            )
            val_sequences, val_masks, val_lengths = self._vectorize_texts(
                val_texts_clean, preprocessor.vocab_builder, preprocessor.max_length
            )
            test_sequences, test_masks, test_lengths = self._vectorize_texts(
                test_texts_clean, preprocessor.vocab_builder, preprocessor.max_length
            )
            
            # 8. 创建数据集
            print("🗂️ 创建完整数据集...")
            datasets = self._create_datasets(
                train_sequences, train_masks, train_labels, train_lengths,
                val_sequences, val_masks, val_labels, val_lengths,
                test_sequences, test_masks, test_labels, test_lengths
            )
            
            # 9. 准备保存数据
            vocab_info = {
                'token_to_id': preprocessor.vocab_builder.token_to_id,
                'id_to_token': preprocessor.vocab_builder.id_to_token,
                'vocab_size': preprocessor.vocab_builder.vocab_size,
                'special_tokens': preprocessor.vocab_builder.special_tokens
            }
            
            metadata = {
                'sequence_length_analysis': {
                    'mean_length': np.mean(train_lengths),
                    'max_length': np.max(train_lengths),
                    'min_length': np.min(train_lengths),
                    'total_samples': len(train_sequences)
                },
                'class_distribution': {
                    'train': dict(Counter(train_labels)),
                    'val': dict(Counter(val_labels)),
                    'test': dict(Counter(test_labels))
                }
            }
            
            # 10. 保存数据
            DataSaver.save_processed_data_multiclass(datasets, vocab_info, metadata, './processed_data_multiclass')
            
            print("🎉 完整数据处理完成！")
            return True
            
        except Exception as e:
            print(f"❌ 处理失败: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def _vectorize_texts(self, texts: List[str], vocab_builder: VocabularyBuilder, max_length: int):
        """向量化文本数据"""
        sequences, masks, lengths = [], [], []
        
        for i, text in enumerate(texts):
            seq, mask, length = vocab_builder.text_to_sequence(text, max_length)
            sequences.append(seq)
            masks.append(mask)
            lengths.append(length)
            
            if i % 5000 == 0 and i > 0:
                print(f"  已向量化 {i}/{len(texts)} 个样本")
        
        return sequences, masks, lengths
    
    def _create_datasets(self, train_sequences, train_masks, train_labels, train_lengths,
                        val_sequences, val_masks, val_labels, val_lengths,
                        test_sequences, test_masks, test_labels, test_lengths):
        """创建数据集"""
        train_dataset = IMDBDataset(train_sequences, train_masks, train_labels, train_lengths)
        val_dataset = IMDBDataset(val_sequences, val_masks, val_labels, val_lengths)
        test_dataset = IMDBDataset(test_sequences, test_masks, test_labels, test_lengths)
        
        return {
            'train': train_dataset,
            'val': val_dataset,
            'test': test_dataset
        }

## 工具函数

**功能方法：**

检查处理后的数据规模和统计信息

<hr>

In [9]:
def check_data_size(data_path: str = './processed_data_multiclass/all_data.pt'):
    """检查处理了多少数据"""
    try:
        data = torch.load(data_path)
        
        print("\n=== 数据规模检查 ===")
        print(f"训练序列: {data['train_sequences'].shape}")
        print(f"训练标签: {data['train_labels'].shape}")
        print(f"验证序列: {data['val_sequences'].shape}") 
        print(f"测试序列: {data['test_sequences'].shape}")
        
        total_samples = (data['train_sequences'].shape[0] + 
                        data['val_sequences'].shape[0] + 
                        data['test_sequences'].shape[0])
        
        print(f"\n总样本数: {total_samples}")
        
        if total_samples >= 40000:
            print("✅ 处理了完整数据集")
        elif total_samples >= 20000:
            print("⚠️  处理了部分数据集")  
        else:
            print("❌ 数据量异常，请检查处理过程")
            
    except Exception as e:
        print(f"检查失败: {e}")

## 主程序入口

<hr>

In [10]:
if __name__ == "__main__":
    # 创建数据处理流水线
    pipeline = DataProcessingPipeline(
        max_vocab_size=30000,
        max_length=512, 
        min_freq=2,
        seed=42
    )
    
    # 运行完整处理
    success = pipeline.run_pipeline()
    
    if success:
        # 检查数据规模
        check_data_size()
        print("\n🎯 现在可以开始训练模型了！")
    else:
        print("\n❌ 数据处理失败，请检查错误信息")

=== IMDb数据预处理开始 ===
数据目录不完整，重新下载...
下载IMDb数据集...
下载完成，解压中...
IMDb数据集准备完成

📥 加载完整原始数据...
加载IMDb数据集...
train 星级分布: {1: 5100, 2: 2284, 3: 2420, 4: 2696, 7: 2496, 8: 3009, 9: 2263, 10: 4732}
test 星级分布: {1: 5022, 2: 2302, 3: 2541, 4: 2635, 7: 2307, 8: 2850, 9: 2344, 10: 4999}
加载了 25000 训练样本, 25000 测试样本
原始数据: 25000 训练样本, 25000 测试样本
📊 完整数据划分...
执行可复现的数据划分（带类别检查）...
数据划分: 训练集=32000, 验证集=8000, 测试集=10000
训练集 类别分布: {1: 6478, 2: 2935, 3: 3175, 4: 3412, 7: 3074, 8: 3750, 9: 2948, 10: 6228}
验证集 类别分布: {1: 1620, 2: 734, 3: 794, 4: 853, 7: 768, 8: 937, 9: 737, 10: 1557}
测试集 类别分布: {1: 2024, 2: 917, 3: 992, 4: 1066, 7: 961, 8: 1172, 9: 922, 10: 1946}
数据划分: 训练集=32000, 验证集=8000, 测试集=10000
🧹 清洗完整文本数据...
  文本清洗 5000/32000 个样本
  文本清洗 10000/32000 个样本
  文本清洗 15000/32000 个样本
  文本清洗 20000/32000 个样本
  文本清洗 25000/32000 个样本
  文本清洗 30000/32000 个样本
✅ 文本清洗完成
  文本清洗 5000/8000 个样本
✅ 文本清洗完成
  文本清洗 5000/10000 个样本
✅ 文本清洗完成
📚 构建完整词汇表...
构建词汇表...
词汇表构建完成: 30000 个token
词汇表覆盖率: 97.51%
🔢 向量化完整数据...
  已向量化 5000/32000 个样本
  已向量化 1